In [84]:
import os
import cv2
import time
import numpy as np

from skimage import io, filters, color, feature
from sklearn.metrics import jaccard_score

from sklearn.cluster import KMeans
import skimage.exposure as exp
import skimage.morphology as morph

from scipy import ndimage as nd
import matplotlib.pyplot as plt

from multiprocessing import cpu_count
from concurrent.futures import ThreadPoolExecutor


# from LS3_template import evaluate_masks

In [85]:
def read_data(data_dir=os.curdir, folder='train'):

    train_imgs_files = [os.path.join(data_dir, f'{folder}/images', f) for f in
                        sorted(os.listdir(os.path.join(data_dir, f'{folder}/images')))
                        if (os.path.isfile(os.path.join(data_dir, f'{folder}/images', f)) and f.endswith('.jpg'))]

    train_masks_files = [os.path.join(data_dir, f'{folder}/masks', f) for f in
                         sorted(os.listdir(os.path.join(data_dir, f'{folder}/masks')))
                         if (os.path.isfile(os.path.join(data_dir, f'{folder}/masks', f)) and f.endswith('.png'))]
    
    # train_imgs_files.sort()
    # train_masks_files.sort()
    print("Number of train images", len(train_imgs_files))
    print("Number of image masks", len(train_masks_files))
    
    return train_imgs_files, train_masks_files

In [86]:
def plot_imgs(imgs):
    for idx, img in enumerate(imgs):
        plt.subplot(1, len(imgs), idx + 1)
        plt.imshow(img, cmap='binary')
        plt.axis("off")
    plt.show()

In [87]:
def hair_removal(image):  # blackhat filtering
    blackhat = cv2.morphologyEx(
        cv2.cvtColor(image, cv2.COLOR_RGB2GRAY),
        cv2.MORPH_BLACKHAT,
        kernel=cv2.getStructuringElement(1, (3, 3))
    )
    _, threshold = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    filtered_image = cv2.inpaint(image, threshold, 1, cv2.INPAINT_TELEA)
    return filtered_image

In [88]:
def pre_processing(image):
    
    filtered_image = hair_removal(image)
    
    filtered_image = color.rgb2hsv(filtered_image)[:, :, 1]  # saturation channel
    
    filtered_image = filters.median(filtered_image, selem=morph.disk(20))
    filtered_image = filters.rank.geometric_mean(filtered_image, selem=morph.disk(20))
    filtered_image = filters.gaussian(filtered_image, sigma=11, multichannel=True)
    
    filtered_image = exp.adjust_log(filtered_image, 1)  # logarithmic correction

    return filtered_image

In [89]:
def threshold_segmentation(image, method='isodata'):
    
    threshold = thresholding(image, method)
    
    predicted_mask = (image > threshold).astype('int')
    
    return predicted_mask

In [90]:
def thresholding(image, method):
    
    if method == 'otsu':
        threshold = filters.threshold_otsu(image)
    elif method == 'isodata':
        threshold = filters.threshold_isodata(image)
    elif method == 'yen':
        threshold = filters.threshold_yen(image)
    elif method == 'mean':
        threshold = filters.threshold_mean(image)
    elif method == 'triangle':
        threshold = filters.threshold_triangle(image)
    elif method == 'li':
        threshold = filters.threshold_li(image)
    else:
        raise Exception('Unknown method')
    
    return threshold

In [91]:
def cluster_segmentation(image, k=2):
    h, w = image.shape
    kmeans = KMeans(n_clusters=k, random_state=0).fit(image.reshape(h * w, 1))
    
    cluster_centers = kmeans.cluster_centers_
    lower_level = min(np.unique(cluster_centers))
    
    predicted_mask_levels = cluster_centers[kmeans.labels_].reshape(h, w)
    predicted_mask = np.where(predicted_mask_levels == lower_level, 0, 1)
    
    return predicted_mask

In [92]:
def post_processing(image):
    
    filtered_image = nd.binary_fill_holes(image)
    filtered_image = morph.erosion(filtered_image, selem=morph.disk(5))
    filtered_image = morph.dilation(filtered_image, selem=morph.disk(30))
    
    return filtered_image

In [93]:
def skin_lesion_segmentation(img_root):
    """
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %                                                                       %
    %  Template for implementing the main function of the segmentation      %
    % system: 'skin_lesion_segmentation'                                    %
    % - Input: file name of the image showing the skin lesion               %
    % - Output: predicted segmentation mask                                 %
    %                                                                       %
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    SKIN_LESION_SEGMENTATION:
    - - -  COMPLETE - - -
    """
    # Code for the BASELINE system
    image = io.imread(img_root)
    filtered_image = pre_processing(image)
    
    predicted_mask = threshold_segmentation(filtered_image, method='isodata')  # isodata
    # predicted_mask = cluster_segmentation(filtered_image, k=2)

    predicted_mask = post_processing(predicted_mask)
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    return predicted_mask

In [94]:
def evaluate_masks(img_roots, gt_masks_roots):
    """ EVALUATE_MASKS: 
        It receives two lists:
        1) A list of the file names of the images to be analysed
        2) A list of the file names of the corresponding Ground-Truth (GT) 
            segmentations
        For each image on the list:
            It performs the segmentation
            It determines Jaccard Index
        Finally, it computes an average Jaccard Index for all the images in 
        the list
    """
    score = []
    for i in np.arange(np.size(img_roots)):
        print('\nI%d' % i)
        predicted_mask = skin_lesion_segmentation(img_roots[i])
        gt_mask = io.imread(gt_masks_roots[i]) / 255
        jacc_score = jaccard_score(np.ndarray.flatten(gt_mask), np.ndarray.flatten(predicted_mask))
        print('jacc_score', jacc_score)
        score.append(jacc_score)
        
    mean_score = np.mean(score)
    print(f'Average Jaccard Index: {mean_score}')
    return mean_score

In [95]:
def compute_jacc_score(img, gt_mask):
    predicted_mask = skin_lesion_segmentation(img)
    gt_mask = io.imread(gt_mask) / 255
    jacc_score = jaccard_score(np.ndarray.flatten(gt_mask), np.ndarray.flatten(predicted_mask))
    return jacc_score

In [96]:
def parallel_evaluation(img_roots, gt_masks_roots, plot=False):
    with ThreadPoolExecutor(max_workers=cpu_count()) as pool:
        futures = [
            pool.submit(
                compute_jacc_score,
                img_roots[i], gt_masks_roots[i]
            )
            for i in np.arange(np.size(img_roots))
        ]
        
        scores = [f.result() for f in futures if f.result()]
    
    mean_score = np.mean(scores)
    return mean_score

In [97]:
def main():
    st = time.time()
    train_imgs_files, train_masks_files = read_data(folder='train')
    
#     mean_score = evaluate_masks(train_imgs_files, train_masks_files)
    mean_score = parallel_evaluation(
        tuple(train_imgs_files), tuple(train_masks_files)
    )
    print(f'Mean Jacc score: {mean_score}')
    print(f'Elapsed time: {time.time() - st}')

if __name__ == '__main__':
    main()

Number of train images 50
Number of image masks 50


C:\Users\sergi\AppData\Local\Temp/ipykernel_34268/2939500199.py:17: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  filtered_image = pre_processing(image)
C:\Users\sergi\AppData\Local\Temp/ipykernel_34268/2939500199.py:17: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  filtered_image = pre_processing(image)
C:\Users\sergi\AppData\Local\Temp/ipykernel_34268/2939500199.py:17: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  filtered_image = pre_processing(image)
C:\Users\sergi\AppData\Local\Temp/ipykernel_34268/2939500199.py:17: UserWarning: Possible precision loss converting image of ty

Mean Jacc score: 0.8204984560473556
Elapsed time: 84.42292094230652
